In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import json
from typing import Tuple
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.models import load_model
from tensorflow.keras import layers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataJSON  = []
with open("/content/drive/MyDrive/Colab Notebooks/apt/df_pp_fasttext.json", 'r') as f:
  dataJSON=json.load(f)
  df_pp_fasttext = pd.DataFrame(dataJSON)

df_pp_fasttext["allVectors"] = df_pp_fasttext["query_vector"] + df_pp_fasttext["product_vector"] 
df_pp_fasttext['allVectors']=df_pp_fasttext['allVectors'].apply(lambda x : np.asarray(x,np.float))

df_pp_fasttext["median_relevance"] = df_pp_fasttext["median_relevance"] - 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [4]:
for i in df_pp_fasttext["allVectors"]:
  print(i.shape)
print(df_pp_fasttext["allVectors"].shape)

Streaming output truncated to the last 5000 lines.
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600,)
(600

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_pp_fasttext['allVectors'], df_pp_fasttext['median_relevance'],test_size=0.5, random_state=42, stratify=df_pp_fasttext['median_relevance']) 
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

X_tr_ind=X_train.index.to_numpy()
X_tst_ind=X_test.index.to_numpy()
X_val_ind=X_val.index.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

X_test=np.vstack(X_test)
X_train=np.vstack(X_train)
X_val=np.vstack(X_val)

In [6]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)
y_val = to_categorical(y_val, 4)

In [7]:
model = K.Sequential()
model.add(layers.Embedding(input_dim=600, output_dim=64))
model.add(layers.LSTM(128))
model.add(layers.Dense(4))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          38400     
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 4)                 516       
                                                                 
Total params: 137,732
Trainable params: 137,732
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile('adam','categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,y_train,epochs=20,batch_size=2048, validation_data=(X_val,y_val))

Epoch 1/20
3/3 [==============================] - 76s 22s/step - loss: 5.8015 - accuracy: 0.1555 - val_loss: 5.2754 - val_accuracy: 0.1441
Epoch 2/20
3/3 [==============================] - 63s 20s/step - loss: 5.2237 - accuracy: 0.1453 - val_loss: 5.2687 - val_accuracy: 0.1441
Epoch 3/20
3/3 [==============================] - 63s 20s/step - loss: 5.2065 - accuracy: 0.1453 - val_loss: 5.2426 - val_accuracy: 0.1441
Epoch 4/20
3/3 [==============================] - 65s 21s/step - loss: 5.1797 - accuracy: 0.1453 - val_loss: 5.2222 - val_accuracy: 0.1441
Epoch 5/20
3/3 [==============================] - 68s 23s/step - loss: 5.1591 - accuracy: 0.1453 - val_loss: 5.2061 - val_accuracy: 0.1441
Epoch 6/20
3/3 [==============================] - 74s 21s/step - loss: 5.1433 - accuracy: 0.1453 - val_loss: 5.1923 - val_accuracy: 0.1441
Epoch 7/20
3/3 [==============================] - 66s 22s/step - loss: 5.1287 - accuracy: 0.1479 - val_loss: 5.1796 - val_accuracy: 0.1760
Epoch 8/20
3/3 [===========

In [9]:
model.evaluate(X_test,y_test)

80/80 [==============================] - 12s 155ms/step - loss: 5.0405 - accuracy: 0.1662


[5.0404744148254395, 0.16620716452598572]